

https://www.kaggle.com/christofhenkel/keras-baseline

In [198]:
#########################################################################
# CURRENTLY WORKING WITH TENSORFLOW
# 
# PRIORITIES:
# 1. refactor code to better groupings
# 2. where do the functions go
# 3. grab validation data
# 4. test testing data generator

#########################################################################


# ====================================== #
### CURRENT WORKING IMAGE INPUT PARAMS ###

# IMAGES_PER_BATCH
# original images loaded and vstacked
# 16 images = 23 GB
# 8 images is faster

# MODEL_BATCH_SIZE
# how many of the splitter output arrays to process at one time
# 128 = ERROR
# 96 is okay

# SPLITS
# row X col = total images into array
# more splits = larger batch size

### GOOD ###
images_per_batch = 16
split_rows = 20
split_cols = 20
model_batch_size = 96
augmentation = False

### NOT GOOD ###
images_per_batch = 16
split_rows = 20
split_cols = 20
model_batch_size = 96
augmentation = True
# with augmentation 28 GB used memory, going into swap

### NOT GOOD ###
images_per_batch = 16
split_rows = 24
split_cols = 24
model_batch_size = 128
augmentation = True
# batch size too large to fit in GPU


# ====================================== #



In [200]:
# # ======================================== FOR RUNNING ON THE MACBOOK PRO ====================

### TODO ###
# Check for MacOS environment to enable this automatically

# ## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
# import plaidml.keras
# plaidml.keras.install_backend()

# # Help MacOS be able to use Keras
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# # Gets rid of the processor warning.
# os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [201]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from callbacks import ValPlotCallback
from model import make_model
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from tqdm.keras import TqdmCallback
from tqdm import tqdm_notebook as tqdm

import numpy as np
import glob
import cv2
import os
from pathlib import Path
from itertools import islice

%matplotlib inline

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
train_image_dir = 'images/training/build/0/'
train_mask_dir = 'images/training/mask/0/'
test_image_dir = 'images/training/test/0/'
data_dir = 'data/'


In [193]:
SEED = 77
train_val_split_size = 0.1
images_per_batch = 8  # decrease this number if running out of memory

# ======Image Splitter Params ====== #
split_rows = 20
split_cols = 20
resize = True
image_resize_width = 4800
image_resize_height = 4800

# ======================= MODEL PARAMS ======================= #
epochs = 1
model_batch_size = 64
model_name = 'datagenmodel'
model_path = os.path.join(data_dir, model_name + '.h5')
pretrained_model = False
pretrained_model_path = model_path
print_model_summary_on_compile = False
plot_epoch_val_images = True

# ==================== Data Augmentation ==================== #
data_augmentation = True
datagen_args = dict(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    # randomly rotate images in the range (degrees, 0 to 180)
    rotation_range=60,
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.2,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.2,
    shear_range=0.0,  # set range for random shear
    zoom_range=0.0,  # set range for random zoom
    channel_shift_range=0.0,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.0,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format='channels_last',
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0,
)

# ==================== Callbacks ==================== #
early_stop = EarlyStopping(patience=5, verbose=1)
check_point = ModelCheckpoint(
    os.path.join(data_dir, model_name + '.hdf5'), save_best_only=True, verbose=1
)
tensor_board = TensorBoard(
    log_dir='../logs/tensorboard/',
    histogram_freq=1,
    write_graph=True,
    write_grads=False,
    write_images=True,
    embeddings_freq=1,
    update_freq='epoch',
)


In [204]:
def array_properties(name, array):
    """Prints the length shape and memory size of an array"""
    print('==========')
    print(name.upper())
    print(f'Length: {len(array)}')
    print(f'Shape: {array.shape}')
    print(f'Size: {round(array.itemsize * array.size / 1024 / 1024 / 1024, 3)} GB')
    print('==========')
    print('')

In [180]:
def create_batches(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [196]:
def batch_stacker(batches):
    for batch in batches:
        array = [cv2.imread(str(image)).astype(np.uint8) for image in batch]
        stack = (np.vstack(array)/255).astype(np.float32)
        yield stack

In [197]:
train = Path(train_image_dir).glob('*')
mask = Path(train_mask_dir).glob('*')
train = list(train)[:4]
mask = list(mask)[:4]

train_img_batches = create_batches(train, 2)
train_mask_batches = create_batches(mask, 2)

for train_stack, mask_stack in zip(
    batch_stacker(train_img_batches), batch_stacker(train_mask_batches), strict=True
):
    # print('train_stack')
    # array_properties(train_stack)
    # print('mask stack')
    # array_properties(mask_stack)
    x_train, x_val, y_train, y_val = train_test_split(
        train_stack, mask_stack, random_state=SEED, test_size=train_val_split_size
    )
    # print('xtrain')
    # array_properties(x_train)
    # print('ytrain')
    # array_properties(y_train)
    # print('xval')
    # array_properties(x_val)
    # print('yval')
    # array_properties(y_val)
    # print('-----===================-----------------')
    # print()

    print('Creating New Model')
    inputs = Input((x_train.shape[1], x_train.shape[2], 3))
    model = make_model(inputs=inputs, model_name=model_name, print_summary=True)

    # =================== MODEL PARAMS =================== #
    validation_plots = ValPlotCallback(model, model_batch_size, x_val, y_val)
    model_fit_params = dict(
        batch_size=model_batch_size,
        epochs=epochs,
        validation_data=(x_val, y_val),
        verbose=1,
        steps_per_epoch=x_train.shape[0] // model_batch_size,
        validation_steps=(x_train.shape[0] // model_batch_size) * train_val_split_size,
        callbacks=[early_stop, check_point, tensor_board, validation_plots],
    )

    print('Augmenting Data')

    # provide the same seed and keyword arguments
    image_datagen = ImageDataGenerator(**datagen_args)
    mask_datagen = ImageDataGenerator(**datagen_args)

    image_datagen.fit(x_train, augment=True, seed=SEED)
    mask_datagen.fit(y_train, augment=True, seed=SEED)

    # save_to_dir='../images/augmented/images/',
    # save_to_dir='../images/augmented/masks/',
    image_generator = image_datagen.flow(x_train, seed=SEED)
    mask_generator = mask_datagen.flow(y_train, seed=SEED)
    ### NOTE ###
    # Can I use the validation setting on the image generators and get rid of the train/test split
    # then I can feed my images in as generators to the generators and skip the loop

    # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)

    model.fit(train_generator, **model_fit_params)

model.save(model_path)


Creating New Model


2022-06-01 16:51:31.606281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Exception encountered when calling layer "max_pooling2d_1" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_1/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,2500,1,32].

Call arguments received by layer "max_pooling2d_1" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 2500, 1, 32), dtype=float32)

In [ ]:

    # ================================================================ #
    # =========================== TRAINING =========================== #
    
    # load or train model
    if pretrained_model or batch_number > 1:
        print('Loading Trained Model')
        model = load_model(trained_model)
    else:
        print('Creating New Model')
        inputs = Input((x_train.shape[1], x_train.shape[2], 3))
        model = make_model(inputs=inputs,
                           model_name=model_name)
        
    # =================== MODEL PARAMS =================== #
    model_fit_params = dict(batch_size=model_batch_size,
                        epochs=epochs,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        steps_per_epoch=x_train.shape[0] // model_batch_size,
                        validation_steps=(x_train.shape[0] // model_batch_size) * train_val_split_size,
                        callbacks=[early_stop, check_point, tensor_board, validation_plots])
        
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train, **model_fit_params)
    else:
        print('Using real-time data augmentation.')

        # provide the same seed and keyword arguments
        image_datagen = ImageDataGenerator(**datagen_args)
        mask_datagen = ImageDataGenerator(**datagen_args)

        image_datagen.fit(x_train, augment=True, seed=seed)
        mask_datagen.fit(y_train, augment=True, seed=seed)

        # save_to_dir='../images/augmented/images/',
        # save_to_dir='../images/augmented/masks/',
        image_generator = image_datagen.flow(x_train, seed=seed)
        mask_generator = mask_datagen.flow(y_train, seed=seed)
        ### NOTE ###
        # Can I use the validation setting on the image generators and get rid of the train/test split
        # then I can feed my images in as generators to the generators and skip the loop
        
        # combine generators into one which yields image and masks
        train_generator = zip(image_generator, mask_generator)

        model.fit(train_generator, **model_fit_params)

    model.save(model_path)

In [ ]:
model.save(model_path)

In [ ]:
# ================================================================ #
# ========================== VALIDATION ========================== #

x_val_pred = model.predict(x_val, verbose=1, batch_size=model_batch_size)

model.evaluate(x=x_val, y=y_val, batch_size=model_batch_size)

# simple threshold to change to 1/0, mask
x_val_pred_mask = (x_val_pred > 0.5).astype(np.uint8)

In [ ]:
plot_predictions(original=x_val, predicted=x_val_pred,
                 predicted_mask=x_val_pred_mask, ground_truth=y_val)

In [ ]:
# model = load_model('../data/testing_model.h5')

In [ ]:
# ================================================================ #
# ========================== PREDICTION ========================== #


x_test = [np.array(
    image_splitter(
        cv2.imread(test_image_dir + img_name).astype(np.uint8),
        num_col_splits=split_cols,
        num_row_splits=split_rows,
        resize=resize,
        resize_height=image_resize_height,
        resize_width=image_resize_width
    )
) for img_name in tqdm(test_filenames[:20])]

x_test = (np.vstack(x_test)/255).astype(np.float32)

shape_and_mem(x_test)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(x_test, seed=seed)

y_pred = model.predict(test_generator, verbose=1)

shape_and_mem(y_pred)

y_pred_mask = (y_pred > 0.5).astype(np.uint8)

In [ ]:
plot_predictions(original=x_test, predicted=y_pred,
                 predicted_mask=y_pred_mask)


# https://www.jeremyjordan.me/evaluating-image-segmentation-models/

# result = cv2.bitwise_and(test_split[0], test_split[0], mask=prediction[0])

# result

In [ ]:
# https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2

# https://www.jeremyjordan.me/evaluating-image-segmentation-models/

